In [1]:
print('Ready.')

Ready.


In [2]:
import os
import importlib

import chex
import jax
import numpy as np
import pickle
import requests

In [27]:
DATA_DIR = '../data/tiny_shakespeare'
DATASET_PATH = os.path.join(DATA_DIR, 'tiny_shakespeare.txt')
TRAIN_SPLIT_SIZE = 0.9

BATCH_SIZE = 5
SEQLEN = 7

## Dataset

Adapted from https://github.com/karpathy/nanoGPT/blob/master/data/shakespeare_char/prepare.py.

In [4]:
if not os.path.exists(DATASET_PATH):
    url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(DATASET_PATH, 'w') as f:
        f.write(requests.get(url).text)


with open(DATASET_PATH, 'r') as f:
    raw_data = f.read()

In [5]:
chars = sorted(list(set(raw_data)))
vocab_size = len(chars)

print(f'Vocab size: {vocab_size}')
print(f"Vocab: {chars}")

Vocab size: 65
Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
# Create mappings between characters and integers.
stoi = { char: i for i, char in enumerate(chars) }
itos = { i: char for i, char in enumerate(chars) }

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    ''.join([itos[i] for i in l])

In [7]:
# Create the train and test splits.
n = len(raw_data)
train_data = raw_data[:int(n * TRAIN_SPLIT_SIZE)]
val_data = raw_data[int(n * TRAIN_SPLIT_SIZE):]

In [8]:
# Encode both to integers.
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"Train split has {len(train_ids):,} tokens")
print(f"Val split has {len(val_ids):,} tokens")

Train split has 1,003,854 tokens
Val split has 111,540 tokens


In [9]:
# Export to bin files.
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(DATA_DIR, 'train.bin'))
val_ids.tofile(os.path.join(DATA_DIR, 'val.bin'))

In [10]:
# Save the meta information as well, to help us encode and decode later.
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(os.path.join(DATA_DIR, 'meta.pkl'), 'wb') as f:
    pickle.dump(meta, f)

In [11]:
@chex.dataclass
class Batch:
    inputs: chex.ArrayDevice
    targets: chex.ArrayDevice

In [12]:
train_path = os.path.join(DATA_DIR, 'train.bin')
val_path = os.path.join(DATA_DIR, 'val.bin')

train_data = np.memmap(train_path, dtype=np.uint16, mode='r')
val_data = np.memmap(val_path, dtype=np.uint16, mode='r')

def get_batch(split) -> Batch:
    assert split in ['train', 'val']
    data = train_data if split == 'train' else val_data
    idx = np.random.randint(
        low=0,
        high=len(data) - SEQLEN,
        size=(BATCH_SIZE,),
    )
    inputs = np.stack([data[i:i + SEQLEN] for i in idx]).astype(np.int64)
    targets = np.stack([
        data[i + 1:i + 1 + SEQLEN] for i in idx
    ]).astype(np.int64)
    return Batch(inputs=inputs, targets=targets)

In [13]:
batch = get_batch('train')
print('Inputs:', batch.inputs.shape, batch.inputs.dtype)
print('Targets:', batch.targets.shape, batch.inputs.dtype)

Inputs: (5, 7) int64
Targets: (5, 7) int64


## Training a small Transformer

In [91]:
from jax import numpy as jnp

import fax
from fax import modules

for module in [
    fax.initializers,
    fax.modules.utils,
    fax.modules.transformer.transformer,
    fax.modules.transformer.config,
    fax.modules,
]:
    importlib.reload(module)

In [78]:
key = jax.random.PRNGKey(seed=0)
cfg = modules.transformer.config.TransformerConfig(
    d_model=8,
    d_head=12,
    d_ff=16,
    n_heads=4,
    vocab_size=vocab_size,
    n_layers=1,
    max_len=SEQLEN,
)
transformer = modules.Transformer(key=key, cfg=cfg)

### jit

In [85]:
%timeit transformer(batch.inputs).block_until_ready()

5 ms ± 43.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [86]:
transformer_jit = jax.jit(transformer)
transformer_jit(batch.inputs)  # compile
%timeit transformer_jit(batch.inputs).block_until_ready()

11.5 µs ± 82.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### vmap

In [93]:
N_DEVICES = 2

multi_inputs = np.broadcast_to(batch.inputs[None, ...], (N_DEVICES, BATCH_SIZE, SEQLEN))
multi_targets = np.broadcast_to(batch.targets[None, ...], (N_DEVICES, BATCH_SIZE, SEQLEN))
multi_batch = Batch(inputs=multi_inputs, targets=multi_targets)

In [96]:
transformer_vmap = jax.vmap(transformer)
transformer_vmap(multi_batch.inputs).shape

(2, 5, 7, 65)